In [1]:
from transformers import DetrImageProcessor
import cv2

/Users/sudipkhadka/Desktop/Object_Detection/.obj/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import supervision as sv
import torch
from transformers import DetrForObjectDetection

In [3]:
image_processor = DetrImageProcessor.from_pretrained('facebook/detr-resnet-50')
box_annot = sv.BoxAnnotator()

In [4]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model1 = DetrForObjectDetection.from_pretrained('/Users/sudipkhadka/Desktop/Object_Detection/Research/trained_model')
model1 = model1.to(DEVICE)

In [5]:
import easyocr

In [6]:

img_reader = easyocr.Reader(['en'], gpu=False)
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('Error: Could not access camera')
    exit()

print("Searching for License Plates........ Press 'q' to exit")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print('Error: Could not read frame')
            break

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        with torch.no_grad():
            input = image_processor(images=img, return_tensors='pt').to(DEVICE)
            outputs = model1(**input)

            CONFIDENCE_THRESHOLD = 0.85
            target_size = torch.tensor([img.shape[:2]]).to(DEVICE)
            results = image_processor.post_process_object_detection(
                outputs=outputs, threshold=CONFIDENCE_THRESHOLD, target_sizes=target_size
            )[0]

            detections = sv.Detections.from_transformers(transformers_results=results)

            for box in detections.xyxy:
                x_min, y_min, x_max, y_max = map(int, box)
                plate_img = frame[y_min:y_max, x_min:x_max]  # Extract the plate in BGR format
                
                # Perform OCR on the detected license plate
                plate_text = img_reader.readtext(plate_img, detail=0)  # Extract text only
                
                if plate_text:  # Ensure OCR detected some text
                    detected_text = plate_text[0]  # Get first detected text
                    print(f"Detected Plate: {detected_text}")

                    text_position = (x_min, max(y_min - 10, 20))  # Display above the box
                    cv2.putText(
                        frame, f"Plate: {detected_text}", text_position,
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA
                    )
            
            frame = box_annot.annotate(scene=frame, detections=detections) 

            cv2.imshow('License Plate Detection', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
finally:
    cap.release()
    cv2.destroyAllWindows()
    print('Camera Released and all windows closed')

'\nimg_reader = easyocr.Reader([\'en\'], gpu=False)\ncap = cv2.VideoCapture(0)\nif not cap.isOpened():\n    print(\'Error: Could not access camera\')\n    exit()\n\nprint("Searching for License Plates........ Press \'q\' to exit")\n\ntry:\n    while True:\n        ret, frame = cap.read()\n        if not ret:\n            print(\'Error: Could not read frame\')\n            break\n\n        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)\n        \n        with torch.no_grad():\n            input = image_processor(images=img, return_tensors=\'pt\').to(DEVICE)\n            outputs = model1(**input)\n\n            CONFIDENCE_THRESHOLD = 0.85\n            target_size = torch.tensor([img.shape[:2]]).to(DEVICE)\n            results = image_processor.post_process_object_detection(\n                outputs=outputs, threshold=CONFIDENCE_THRESHOLD, target_sizes=target_size\n            )[0]\n\n            detections = sv.Detections.from_transformers(transformers_results=results)\n\n            for b